In [1]:
"""
# Bug Detection and Fixing with LLM

This Jupyter Notebook takes user-input Python code, classifies it as 'buggy' or 'bug-free' using a fine-tuned LLM, and suggests fixes.
"""

"\n# Bug Detection and Fixing with LLM\n\nThis Jupyter Notebook takes user-input Python code, classifies it as 'buggy' or 'bug-free' using a fine-tuned LLM, and suggests fixes.\n"

In [2]:
!pip install torch transformers datasets jupyter pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
uploaded = files.upload()

Saving github_scraped.csv to github_scraped.csv


In [9]:
csv_filename = "github_scraped.csv" # got from the scrape data set from github

In [10]:
# Load the CSV file into a pandas DataFrame
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame({"code": [], "label": []})

In [11]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1)  # 10% for validation

In [12]:
# Convert the train and validation DataFrames into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [13]:
# Load pre-trained model and tokenizer
MODEL_NAME = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Inspect the dataset columns
print(train_dataset.column_names)
print(val_dataset.column_names)

# Check unique labels in the dataset to update label mapping
print(set(train_dataset['label']))

# Function to tokenize the code
def tokenize_function(examples):
    return tokenizer(examples['code'], padding=True, truncation=True, max_length=512)

# Apply tokenization to both train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Update the label map according to your actual dataset labels
label_map = {'bug-free': 0, 'buggy': 1}

# Apply the label mapping
train_dataset = train_dataset.map(lambda x: {'label': label_map[x['label']]})
val_dataset = val_dataset.map(lambda x: {'label': label_map[x['label']]})

# Now create a DatasetDict with both 'train' and 'validation' splits
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

['code', 'label', '__index_level_0__']
['code', 'label', '__index_level_0__']
{'bug-free', 'buggy'}


Map:   0%|          | 0/430 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/430 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [15]:
# Define compute_metrics function to calculate accuracy
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=1)
    return {"eval_accuracy": accuracy_score(labels, preds)}

def train_model():
    # Adjusted training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        logging_steps=10,
        learning_rate=1e-6,
        per_device_train_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
        warmup_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        greater_is_better=True
    )

    # Trainer with compute_metrics function
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

In [16]:
# Train the model
train_model()

<ipython-input-15-0a3383fab177>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: debayanghosh408 (debayanghosh408-brainware-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.707700,0.717868,0.458333
2,0.707100,0.715083,0.458333


In [17]:
import ast
import traceback

def classify_code(code):
    # Try parsing the code to detect syntax errors
    try:
        ast.parse(code)
        return "bug-free", None
    except SyntaxError as e:
        return "buggy", str(e)

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

FIX_MODEL_NAME = "Salesforce/codet5-small"  # Change this if using a different CodeT5 model
fix_tokenizer = AutoTokenizer.from_pretrained(FIX_MODEL_NAME)
fix_model = AutoModelForSeq2SeqLM.from_pretrained(FIX_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [22]:
def fix_buggy_code(code):
    """Generates a fixed version of the buggy code using CodeT5."""
    input_text = f"Fix this code: {code}"
    inputs = fix_tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = fix_model.generate(**inputs, max_length=512, num_beams=5, no_repeat_ngram_size=2)  # Added constraints
    fixed_code = fix_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Optionally filter out any non-code parts of the response
    fixed_code = fixed_code.strip()

    return fixed_code

In [25]:
# Example buggy code input
code_input = """
print"Hello, World!")
"""

# Classify the code
status, error = classify_code(code_input)
if status == "buggy":
    print(f"Buggy Code: {error}")
    fixed_code = fix_buggy_code(code_input)
    print("Fixed Code:\n", fixed_code)
else:
    print("Bug-free Code")

Buggy Code: unmatched ')' (<unknown>, line 2)
Fixed Code:
 print "Hello, World!"
